# machince learning tool SVCM, PCA, GBM 
!pip install scikit-learn

In [ ]:
# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
# scikit learn
# from sklearn.datasets import load_iris
import sklearn.datasets as sk_dataset

# split learning / test
from sklearn.model_selection import train_test_split

In [ ]:
# in/out data load

In [ ]:
iris_data = sk_dataset.load_iris()

In [ ]:
iris_x = iris_data.data # input 
iris_y = iris_data.target # output(target)

In [ ]:
type(iris_x), iris_x.shape # sample # 붓꽃의 꽃받침 길이, 꽃받침 너비 과 꽃잎의 길이, 꽃잎의 너비 에 대한 값

In [ ]:
type(iris_y), iris_y.shape, iris_y[100:110] # class number 0- setosa, 1 - versicolor, 2 - virginica

In [ ]:
iris_x[:, 1].min(), iris_x[:, 1].max()  # input-out normalizing 0~1

In [ ]:
# norm 0~1 .. input mapping 
def norm_maxmin(data, out_min=0, out_max=255):
    gradient = (out_max - out_min) / (np.max(data) - np.min(data))
    bias = out_min - np.min(data) * gradient
    out = data * gradient + bias

    return out

In [ ]:
for it_col in range(iris_x.shape[1]):
    iris_x[:, it_col] = norm_maxmin(iris_x[:, it_col], 0, 1)

In [ ]:
iris_x[:, 0].min(), iris_x[:, 0].max()  # input-out normalizing 0~1

In [ ]:
iris_x.shape, iris_y.shape

In [ ]:
# train set, test set
x_train, x_test, y_train, y_test = train_test_split(iris_x, iris_y, test_size=0.1, shuffle=True)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
y_train, y_test

In [ ]:
# mini batch  #norm, mini
batch_size = 15

In [ ]:
x_train.shape # ?, 15, 4

In [ ]:
x_train = x_train.reshape(-1, batch_size, x_train.shape[1])
y_train = y_train.reshape(-1, batch_size, )

In [ ]:
type(x_train), y_train.shape # ?, 15, 4

In [ ]:
# 자료형 
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)

In [ ]:
type(x_train), type(y_train), type(x_test), type(y_test), x_train.dtype, y_train.dtype

# Model init

In [ ]:
class Iris_fcn(nn.Module):
    
    def __init__(self, model_input, model_output):
        super(Iris_fcn, self).__init__()
        
        # layer
        self.fc = nn.Linear(model_input, 16) # learning weight, bias
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, model_output)
        
        #activation 
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        out = self.fc(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        
       # Cross Entropy Loss --> softmax 생략
        
        return out

In [ ]:
model = Iris_fcn(x_train.shape[2], 3)

In [ ]:
from torchsummary import summary

summary(model, input_size=((1, 4)))

# Model train

In [ ]:
learning_rate = 1e-2 #0.01

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
epochs = 1000
for _epoch in range(epochs):
    for _batch in range(x_train.shape[0]):
        optimizer.zero_grad()
        
        output = model(x_train[_batch, ])
        
        # 오차 계산 및 오차 역전파
        loss = criterion(output, y_train[_batch, ])
        loss.backward()

        #기울기 적용
        optimizer.step()
        
    if (_epoch+1) % 10 == 0:
        print(f'epoch:{_epoch}, loss:{loss.item()}')
        
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0

            outputs = model(x_test)
            pred = torch.argmax(outputs, axis=1)

            correct = (pred == y_test).sum()
            total = y_test.shape[0]

            acc = correct/total * 100
            print(f'accuracy : {acc}, correct:{correct}, total{total}')
            
            model.train()

In [ ]:
!pip list | grep torch